# Introduction

Scipp can handle *event data*, a certain type of sparse data, i.e., data that cannot directly be represented as a multi-dimensional array.
For applications that rely solely on dense arrays of data this section can safely be ignored.

Scipp supports event data in shape of a multi-dimensional array of lists.
This is the main application of [Bucketed Data](../user-guide/buckets.ipynb).
This could, e.g., be used to store data from an array of sensors/detectors that are read out independently, with potentially widely varying frequency.

Before assigning events to buckets, we can initialize them as a single long list or table.
In the simplest case this table has just a single column, i.e., it is a scipp variable:

In [ ]:
import numpy as np
import scipp as sc
from scipp.plot import plot

table = sc.Variable(dims=['event'],
                  values=[0,1,3,1,1,1,42,1,1,1,1,1],
                  dtype=sc.dtype.float64)
sc.table(table)

The events in the table can then be mapped into buckets.
Note that this copies `table`:

In [ ]:
begin = sc.Variable(dims=['x'], values=[0,6,6,8])
end = sc.Variable(dims=['x'], values=[6,6,8,12])
var = sc.to_buckets(begin=begin, end=end, dim='event', data=table)
sc.show(var)
var

Each element of the resulting "bucket variable" references a section of the underlying table:

In [ ]:
sc.table(var['x', 0].value)
sc.table(var['x', 1].value)
sc.table(var['x', 2].value)

Operations between variables or data arrays broadcast dense data to lists:

In [ ]:
scale = sc.Variable(dims=['x'], values=np.arange(2.0, 6))
var *= scale
var['x', 0].values

In [ ]:
var['x', 1].values

In [ ]:
var['x', 2].values

In practice events require more than one "column" of information.
Typically we need at least one coordinate such as the detection time in addition to weights.
If each event corresponds to, e.g., a single detected neutron the weight is 1.
As above, we start by creating a single table containing *all* events:

In [ ]:
times = sc.Variable(dims=['event'],
                    unit=sc.units.us, # micro second
                    values=[0,1,3,1,1,1,4,1,1,2,1,1],
                    dtype=sc.dtype.float64)
weights = sc.Variable(dims=['event'],
                      unit=sc.units.counts,
                      values=np.ones(12),
                      variances=np.ones(12),
                      dtype=sc.dtype.float64)

table = sc.DataArray(data=weights, coords={'time':times})
sc.table(table)
table

This table is then mapped into buckets.
The resulting "bucket variable" can, e.g., be used as the data in a data array, and can be combined with coordinates as usual:

In [ ]:
var = sc.to_buckets(begin=begin, end=end, dim='event', data=table)
a = sc.DataArray(data=var, coords={'x':sc.Variable(['x'], values=np.arange(4.0))})
a

In the graphical representation of the data array we can see the dense coordinate (green), and the buckets (yellow):

In [ ]:
sc.show(a)

As before, each bucket references a section of the underlying table:

In [ ]:
sc.table(a['x', 0].value)
sc.table(a['x', 1].value)
sc.table(a['x', 2].value)

# Arithmetic operations

Direct arithmetic operations with event data can be useful in some cases, but more commonly operations should act on event data as if it had been histogrammed are required.
For example, addition of histogrammed data would correspond to concatenating event lists.

Scipp supports such operations for bucket variables.
The following operations are supported:

- "Addition" of data arrays containing event data in buckets.
  This is achieved by concatenating the underlying event lists.
- "Subtraction" of data arrays containing event data in buckets.
  This is performed by concatenating with a negative weight for the subtrahend.
- "Multiplication" of a data array containing event data in buckets with a data array with dense, histogrammed data.
  The weight of each event is scaled by the value of the corresponding bin in the histogram.
- "Division" of a data array containing event data in buckets by a data array with dense, histogrammed data.
  This is performed by scaling with the inverse of the denominator.

<div class="alert alert-warning">
    <b>WARNING:</b>

It is important to note that these operations, in particular multiplication and division, are only interchangeable with histogramming if the variances of the "histogram" operand are negligible.
If these variances are not negligible the operation on the event data introduces correlations in the error bars of the individual events.
Scipp has no way of tracking such correlations and a subsequent `histogram` step propagates uncertainties under the assumption of uncorrelated error bars.
</div>

## Addition

In [ ]:
sc.show(a['x',2].value)
sc.buckets.append(a, a)
sc.show(a['x',2].value)
plot(sc.buckets.sum(a))

## Subtraction

In [ ]:
zero = a.copy()
sc.show(zero['x',2].value)
sc.buckets.append(zero, -zero)
sc.show(zero['x',2].value)
plot(sc.buckets.sum(zero))

## Multiplication and division

In [ ]:
time_bins = sc.Variable(dims=['time'], unit=sc.units.us, values=[0.0, 3.0, 6.0])
sc.buckets.scale(a, sc.histogram(a, time_bins))
plot(sc.histogram(a, time_bins))